In [1]:
import numpy as np
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import os
import csv

In [2]:
# Function to extract data from the problem instances. 

# Input is the file path for the instance ".txt" file

# Outputs are dictionaries containing information with respect to each building, solar panel, battery and activities.

def get_instance_data(instance_path):
    with open(instance_path) as f:
        lines = f.readlines()
    data = []
    for line in lines:
        data.append(line.split())

    ppoi_keys = ['No. of buildings', 'No. of pv panels', 'No. of batteries', 'No. of recurring activities', 
                 'No. of once off activities']

    building_keys = ['No. of small rooms', 'No. of large rooms']

    solar_keys = ['building id']

    battery_keys = ['building id','capacity kWh', 'max power kW', 'efficiency']

    activity_keys = ['No. of small rooms', 'No. of large rooms', 'load kW', 'duration']

    precedence_keys = ['No. of precedences', 'precedence list']

    price_keys = ['value', 'penalty']

    recurring_keys = activity_keys + precedence_keys

    once_off_keys = activity_keys + price_keys + precedence_keys

    ppoi_dict = {}
    
    building_dict = {}

    solar_dict = {}

    battery_dict = {}

    recurring_dict = {}

    once_off_dict = {}

    for line in data:

        if line[0] == 'ppoi':
            ppoi_dict = dict(zip(ppoi_keys, line[1:]))

        if line[0] == 'b':
            if int(line[1]) > 1:
                line[1] = int(line[1]) - 1
            building_dict[int(line[1])] = dict(zip(building_keys, line[2:]))

        if line[0] == 's':
            if int(line[2]) > 1:
                line[2] = int(line[2]) - 1
            solar_dict[int(line[1])] = dict(zip(solar_keys, line[2:]))

        if line[0] == 'c':
            if int(line[2]) > 1:
                line[2] = int(line[2]) - 1
            battery_dict[int(line[1])] = dict(zip(battery_keys, line[2:]))

        if line[0] == 'r':
            if line[3] == 'S':
                line[3] = 0
            else:
                line[3] = int(line[2])
                line[2] = 0
            recurring_dict[int(line[1])] = dict(zip(recurring_keys[:-1], line[2:7]))
            i = 0
            for num in line[7:]:
                line[7 + i] = int(num)
                i = i + 1
            recurring_dict[int(line[1])][recurring_keys[-1]] = line[7:]

        if line[0] == 'a':
            if line[3] == 'S':
                line[3] = 0
            else:
                line[3] = int(line[2])
                line[2] = 0
            once_off_dict[int(line[1])] = dict(zip(once_off_keys[:-1], line[2:9]))
            i = 0
            for num in line[9:]:
                line[9 + i] = int(num)
                i = i + 1
            once_off_dict[int(line[1])][once_off_keys[-1]] = line[9:]

    for item in ppoi_dict:
        ppoi_dict[item] = int(ppoi_dict[item])

    for item in building_dict:
        for elem in building_keys:
            building_dict[item][elem] = int(building_dict[item][elem])

    for item in solar_dict:
        for elem in solar_keys:
            solar_dict[item][elem] = int(solar_dict[item][elem])

    for item in battery_dict:
        for elem in battery_keys:
            if elem != 'efficiency':
                battery_dict[item][elem] = int(battery_dict[item][elem])
            else:
                battery_dict[item][elem] = float(battery_dict[item][elem])

    for item in recurring_dict:
        for elem in recurring_keys:
            if elem != 'precedence list':
                recurring_dict[item][elem] = int(recurring_dict[item][elem])

    for item in once_off_dict:
        for elem in once_off_keys:
            if elem != 'precedence list':
                once_off_dict[item][elem] = int(once_off_dict[item][elem])
    
    return ppoi_dict, building_dict, solar_dict, battery_dict, recurring_dict, once_off_dict, ppoi_keys


In [3]:
# Function to get the predictions for total baseload and total solar generation for the whole month

# Input is the file path for the forecast ".csv" file

# Output are lists containg the total baseload and total solar generation for the whole month 

def get_forecasts(forecast_path):
    data_rows = []
    baseload = []
    solar_power =[]
    with open(forecast_path, 'r') as file:
        csvreader = csv.reader(file)
        for row in csvreader:
            data_rows.append(row)
    for i in range(1, num_points + 1):
        baseload.append(sum(float(data_rows[k][i]) for k in range(6)))
        solar_power.append(sum(float(data_rows[k][i]) for k in range(6, 12)))
        
    return baseload, solar_power


In [4]:
# Function to generate 2 lists : First captures what day of the week it is 1 - Monday, 2- Tuesday and so on at a given time. 

# Second one captures the time instants at which recurring activities 
#must not start (Before first monday, outside office hours and weekends)

# The inputs are starting_day is the day (Mon, Tues and so on) at the start of the month (1st) and start_day 
# is the date of first sunday of the month (1 to 31)

# 44 is to account for the 11 hours of time difference between AEDT and UTC

# The output day_list specifies day at a given time instant

# The output recurring_non_start_list specifies 

def day_list_generator(start_day, starting_day):
    day_list = []
    recurring_non_start_list = []
    j = starting_day - 1
    
    for i in range(num_points + 44):
        if i % 96 == 0:
            j = j + 1
        j = j % 7
        if j == 0:
            j = 7
        day_list.append(j) 
    
    for i in range(num_days + 1):
        for j in range(num_points_day):
            if (j + num_points_day * i < num_points + 44):
                if (j < start_time_sample  or j > stop_time_sample 
                    or day_list[j + num_points_day * i] == 6 or day_list[j + num_points_day * i] == 7 
                    or i < start_day):
                    recurring_non_start_list.append(j + num_points_day * i - 44)
            else:
                break
    
    day_list = day_list[44:]
    recurring_non_start_list = recurring_non_start_list[44:] 
    
    return day_list, recurring_non_start_list

In [5]:
# Function to get the aemo price from ".csv" file

# Input is the general path to the AEMO price folder (Must contain November and December prices)

# Output is a list of AEMO prices at a given time t 

def get_aemo_price(aemo_path):
    aemo_file = ['PRICE_AND_DEMAND_202011_VIC1.csv', 'PRICE_AND_DEMAND_202012_VIC1.csv']
    li = []
    i = 0
    for file in aemo_file:
        price_df = pd.read_csv(aemo_path + '\\' + file)
        li.append(price_df)
    price_df = pd.concat(li, axis=0, ignore_index=True)
    price_df = price_df.loc[price_df.index.repeat(2)]
    price_df['Index'] = [i for i in range(61 * 96)] 
    price_df.set_index('Index', inplace=True)    
    aemo_price = price_df.RRP.to_list()
    aemo_price = aemo_price[40: num_days * 96 + 40]
    return aemo_price

In [6]:
# Function to get Results from Gurobi's optimization problem

# Input is the Gurobi solve object

# Output is the list of decision variables needed for the result file
# recurring_active_time is a list of binary variables which specifies when each recurring activity is active throughout the month
# recurring_start_time is a list of binary variables which specifies when each recurring activity starts throughout the month
# decision is a list of {0, 1, 2} which specifies the battery charge decision for each battery throughout the month

def get_optimization_result(obj):
    var = obj.getVars()
    recurring_activity_status = []
    recurring_start_time = []
    charging_status = []
    discharging_status = []
    decision = []
    for v in var:
        if 'Recurring Activity Status' in v.VarName:
            recurring_activity_status.append(v.X)
        if 'Recurring Start Status' in v.VarName:
            recurring_start_time.append(v.X)
        if 'Battery Charge Status' in v.VarName:
            charging_status.append(v.X)
        if 'Battery Discharge Status' in v.VarName:
            discharging_status.append(v.X)
        if 'Battery Decision' in v.VarName:
            decision.append(v.X)

    recurring_active_time = np.reshape(recurring_activity_status, (num_recurring_activities, num_points))
    recurring_start_time = np.reshape(recurring_start_time, (num_recurring_activities, num_points))
    charging_status = np.reshape(charging_status, (num_batteries, num_points))
    discharging_status = np.reshape(discharging_status, (num_batteries, num_points))
    decision = np.reshape(decision, (num_batteries, num_points))
    
    return recurring_active_time, recurring_start_time, decision 

In [7]:
# Function which defines which rooms are to be assigned to each task

# The input is the building and recurring task information

# Output is the list specifying room usage for each task in each building

def Room_Usage(recurring_dict, building_dict, recurring_start_time):

    for k in range(num_recurring_activities):
        start_time = 0
        for i in range(52, 723):
            start_time = start_time + i * recurring_start_time[k][i]
        recurring_dict[k]['start time'] = start_time
        recurring_dict[k]['end time'] = recurring_dict[k]['start time'] + recurring_dict[k]['duration'] 

    sorted_list = []
    key_list = [i for i in range(num_recurring_activities)]

    for i in range(num_recurring_activities):
        min_no = 10000
        for k in key_list:
            if recurring_dict[k]['start time'] <= min_no:
                min_no = recurring_dict[k]['start time']
                key_value = k 
        sorted_list.append(key_value)
        key_list.remove(key_value)
        
    small_rooms_assigned = [[0 for i in range(num_buildings)] for j in range(num_recurring_activities)]
    large_rooms_assigned = [[0 for i in range(num_buildings)] for j in range(num_recurring_activities)]
    small_rooms_list = [building_dict[k]['No. of small rooms'] for k in range(num_buildings)]
    large_rooms_list = [building_dict[k]['No. of large rooms'] for k in range(num_buildings)]
    
    for i in range(num_recurring_activities):
        small_rooms_available = [small_rooms_list[k] for k in range(num_buildings)]
        large_rooms_available = [large_rooms_list[k] for k in range(num_buildings)]
        small_rooms_needed = recurring_dict[sorted_list[i]]['No. of small rooms']
        large_rooms_needed = recurring_dict[sorted_list[i]]['No. of large rooms']
        for j in range(i):
            if (recurring_dict[sorted_list[i]]['start time'] <= recurring_dict[sorted_list[j]]['end time'] 
            and recurring_dict[sorted_list[i]]['start time'] >= recurring_dict[sorted_list[j]]['start time']):
                small_rooms_available = [x - y 
                                         for x, y in zip(small_rooms_available, small_rooms_assigned[sorted_list[j]][:])]
                large_rooms_available = [x - y 
                                         for x, y in zip(large_rooms_available, large_rooms_assigned[sorted_list[j]][:])]
        for k in range(num_buildings):
            while small_rooms_available[k] > 0 and small_rooms_needed > 0:
                small_rooms_assigned[sorted_list[i]][k] = small_rooms_assigned[sorted_list[i]][k] + 1
                small_rooms_available[k] = small_rooms_available[k] - 1
                small_rooms_needed = small_rooms_needed - 1
            

            while large_rooms_available[k] > 0 and large_rooms_needed > 0:
                large_rooms_assigned[sorted_list[i]][k] = large_rooms_assigned[sorted_list[i]][k] + 1
                large_rooms_available[k] = large_rooms_available[k] - 1
                large_rooms_needed = large_rooms_needed - 1


        total_rooms_assigned = [[0 for i in range(num_buildings)] for j in range(num_recurring_activities)]
        for i in range(num_recurring_activities):
            for k in range(num_buildings):
                total_rooms_assigned[i][k] = large_rooms_assigned[i][k] + small_rooms_assigned[i][k]
        
    return recurring_dict, total_rooms_assigned

In [8]:
# Function to write the solution in the expected format ".txt" file as specified

# Input includes recurring task info, room assignment info, battery decision list and path where solution must be stored

# No Output. The result is stored in the specified path in the required format

def Save_solution(solution_path, recurring_dict, ppoi_dict, total_rooms_assigned, decision):
    ppoi = ['ppoi'] + [str(ppoi_dict[k]) for k in ppoi_keys]
    sched = ['sched', str(num_recurring_activities), '0']
    r = []
    c = []

    for i in range(num_recurring_activities):
        r.append(['r', str(i), str(round(recurring_dict[i]['start time'])), str(recurring_dict[i]['No. of small rooms'] 
                                                                                  + recurring_dict[i]['No. of large rooms'])] 
                + [str(k) if k < 2 else str(k + 1) for k in range(num_buildings) 
                   for j in range(int(total_rooms_assigned[i][k]))])
        
    for i in range(num_batteries):
        for j in range(num_points):
            if decision[i][j] != 1:
                c.append(['c', str(i), str(j), str(int(round(decision[i][j], 1)))])
    lines = [ppoi, sched] + r + c
    with open(solution_path, 'w', encoding='UTF8') as f:
        f.seek(0)
        for line in lines:
            f.writelines(' '.join(line))
            f.write('\n')
        f.truncate()
    return None

In [9]:
# First Sub-Problem defined for Gurobi solver

# Inputs include all details about the instance obtained from get_instance_data function and day_list_generator function

# Output is a Gurobi object which contains all the constraints and the objective function stored, 
# which can be solved outside the function 

def Sub_Problem_1(building_dict, recurring_dict, battery_dict, baseload, 
                      solar_power, day_list, recurring_non_start_list):
    
# Creating Model

    m = gp.Model('Sub Problem 1')
    
# Adding the variables 

    Power_time = m.addVars(num_points, vtype = GRB.CONTINUOUS, lb = 0,  name = 'Total Power')
    battery_power = m.addVars(num_batteries, num_points, vtype = GRB.CONTINUOUS, lb = -GRB.INFINITY, name = 'Battey Power')
    battery_energy = m.addVars(num_batteries, num_points, vtype = GRB.CONTINUOUS, name = 'Battey Energy')
    power_scheduled = m.addVars(num_points, vtype = GRB.CONTINUOUS, lb = 0, name = 'Power Scheduled by activities' )
    battery_charge = m.addVars(num_batteries, num_points, vtype = GRB.BINARY, name = 'Battery Charge Status')
    battery_discharge = m.addVars(num_batteries, num_points, vtype = GRB.BINARY, name = 'Battery Discharge Status')
    battery_decision = m.addVars(num_batteries, num_points, vtype = GRB.CONTINUOUS, name = 'Battery Decision')
    recurring_active_time = m.addVars(num_recurring_activities, 
                                   num_points, vtype = GRB.CONTINUOUS, name = 'Recurring Activity Status')
    recurring_start_time = m.addVars(num_recurring_activities, 
                                   num_points, vtype = GRB.BINARY, name = 'Recurring Start Status')
    max_variable_binary = m.addVars(num_points, vtype=GRB.BINARY, name = 'Max Varible Status')
    max_power = m.addVar(vtype=GRB.CONTINUOUS, name='Maximum Power', lb = 0)

# Maximum Value Constraints

    m.addConstrs(max_power >= Power_time[i] for i in range(num_points))
    
# Battery Power Calculations
    
    m.addConstrs(battery_power[j, i] == 
                 battery_charge[j, i] * battery_dict[j]['max power kW'] * (battery_dict[j]['efficiency'] ** -0.5) - 
                battery_discharge[j, i] * battery_dict[j]['max power kW'] * (battery_dict[j]['efficiency'] ** 0.5)
                for j in range(num_batteries) for i in range(num_points))
    
    m.addConstrs(battery_charge[j, i] + battery_discharge[j, i] <= 1 
                 for j in range(num_batteries) 
                 for i in range(num_points))
    
    m.addConstrs(battery_discharge[j, i] - battery_charge[j, i] + 1 == battery_decision[j, i] 
                 for j in range(num_batteries) 
                 for i in range(num_points))
    
# SOC of battery constraints
    m.addConstrs(battery_energy[j, 0] == battery_dict[j]['capacity kWh'] 
                 + battery_charge[j, 0] * battery_dict[j]['max power kW'] * sample_time_h
                     - battery_discharge[j, 0] * battery_dict[j]['max power kW'] * sample_time_h
                 for j in range(num_batteries))
    
    for i in range(1, num_points):
        m.addConstrs(battery_energy[j, i] <= battery_dict[j]['capacity kWh'] for j in range(num_batteries))
        m.addConstrs(battery_energy[j, i] >= 0 for j in range(num_batteries))
        m.addConstrs(battery_energy[j, i] == battery_energy[j, i - 1] 
                     + battery_charge[j, i] * battery_dict[j]['max power kW'] * sample_time_h
                     - battery_discharge[j, i] * battery_dict[j]['max power kW'] * sample_time_h
                     for j in range(num_batteries))

# Sheduled Power Calculations

    m.addConstrs(power_scheduled[i] == gp.quicksum(recurring_active_time[j, i] * recurring_dict[j]['load kW'] * 
                                                   (recurring_dict[j]['No. of small rooms'] 
                                                    + recurring_dict[j]['No. of large rooms']) 
                                                   for j in range(num_recurring_activities)) 
                 for i in range(num_points))
    
# Power at time i Calculation 
    m.addConstrs(Power_time[i] == baseload[i] - solar_power[i] + power_scheduled[i] + 
                 gp.quicksum(battery_power[j, i] for j in range(num_batteries)) for i in range(num_points))

# Room limit constraints

    m.addConstrs(gp.quicksum(recurring_active_time[j, i] * recurring_dict[j]['No. of small rooms'] 
                             for j in range(num_recurring_activities)) <= gp.quicksum(building_dict[k]['No. of small rooms'] 
                                                                                  for k in range(num_buildings)) 
                 for i in range(num_points))
    
    m.addConstrs(gp.quicksum(recurring_active_time[j, i] * recurring_dict[j]['No. of large rooms'] 
                             for j in range(num_recurring_activities)) <= gp.quicksum(building_dict[k]['No. of large rooms'] 
                                                                                  for k in range(num_buildings)) 
                 for i in range(num_points))

# Start Time constraints

    # Should not start at these times
    
    for num in recurring_non_start_list:
        m.addConstrs(recurring_start_time[j, num] == 0 for j in range(num_recurring_activities))
        m.addConstrs(recurring_active_time[j, num] == 0 for j in range(num_recurring_activities))
    
    # Should start at the same time every week
    
    for num in range(724, num_points):
        m.addConstrs(recurring_start_time[j, num] == recurring_start_time[j, num - 96 * 7] 
                     for j in range(num_recurring_activities))
        m.addConstrs(recurring_active_time[j, num] == recurring_active_time[j, num - 96 * 7] 
                     for j in range(num_recurring_activities))

    # Should happen only once in a week
    
    for j in range(num_recurring_activities):
        m.addConstr(gp.quicksum(recurring_start_time[j, i] 
                                for i in range(52, 723)) == 1)
        m.addConstr(gp.quicksum(recurring_active_time[j, i] 
                                for i in range(52, 723)) 
                    == recurring_dict[j]['duration'] + 1)
        
    # Should last for required duration hours
    
    for j in range(num_recurring_activities):
        m.addConstrs(recurring_start_time[j, i] <= recurring_active_time[j, i + k] 
                     for k in range(recurring_dict[j]['duration'] + 1) 
                     for i in range(num_points - recurring_dict[j]['duration']))
    
    # Precedence Constraints
    
    for j in range(num_recurring_activities):
        if recurring_dict[j]['No. of precedences'] > 0:
            for k in recurring_dict[j]['precedence list']:
                m.addConstr(gp.quicksum(recurring_start_time[j, i] * day_list[i] 
                                        - recurring_start_time[k, i] * day_list[i] 
                                        for i in range(52, 723)) >= 1) 
    
    # Objective Function
    m.setObjective(max_power)
    return m


In [10]:
# Second Sub-Problem defined for Gurobi solver

# Inputs include all details about the instance obtained from get_instance_data function and day_list_generator function. 
# It also includes MipGap at which to solve the problem

# Output is a Gurobi object which contains all the constraints and the objective function stored, 
# which can be solved outside the function 


def Sub_Problem_2(building_dict, recurring_dict, battery_dict, baseload, 
                      solar_power, MipGap, day_list, non_start_list, max_limit):
    
# Creating Model

    m = gp.Model('Sub problem 2')
    
# Adding the variables 

    Power_time = m.addVars(num_points, vtype = GRB.CONTINUOUS, lb = 0,  name = 'Total Power')
    battery_power = m.addVars(num_batteries, num_points, vtype = GRB.CONTINUOUS, lb = -GRB.INFINITY, name = 'Battey Power')
    battery_energy = m.addVars(num_batteries, num_points, vtype = GRB.CONTINUOUS, name = 'Battey Energy')
    power_scheduled = m.addVars(num_points, vtype = GRB.CONTINUOUS, lb = 0, name = 'Power Scheduled by activities' )
    battery_charge = m.addVars(num_batteries, num_points, vtype = GRB.BINARY, name = 'Battery Charge Status')
    battery_discharge = m.addVars(num_batteries, num_points, vtype = GRB.BINARY, name = 'Battery Discharge Status')
    battery_decision = m.addVars(num_batteries, num_points, vtype = GRB.CONTINUOUS, name = 'Battery Decision')
    recurring_active_time = m.addVars(num_recurring_activities, 
                                   num_points, vtype = GRB.CONTINUOUS, lb = 0, ub = 1, name = 'Recurring Activity Status')
    recurring_start_time = m.addVars(num_recurring_activities, 
                                   num_points, vtype = GRB.BINARY, name = 'Recurring Start Status')
    max_variable_binary = m.addVars(num_points, vtype=GRB.BINARY, name = 'Max Varible Status')
    max_power = m.addVar(vtype=GRB.CONTINUOUS, name='Maximum Power', lb = 0)

# Maximum Value Constraints

    m.addConstrs(max_power >= Power_time[i] for i in range(num_points))
    
# Maximum Limit on power scheduled at each time instant constraints

    m.addConstrs(max_limit >= Power_time[i] for i in range(num_points))
    
    
# Battery Power Calculations
    
    m.addConstrs(battery_power[j, i] == 
                 battery_charge[j, i] * battery_dict[j]['max power kW'] * (battery_dict[j]['efficiency'] ** -0.5) - 
                battery_discharge[j, i] * battery_dict[j]['max power kW'] * (battery_dict[j]['efficiency'] ** 0.5)
                for j in range(num_batteries) for i in range(num_points))
    
    m.addConstrs(battery_charge[j, i] + battery_discharge[j, i] <= 1 
                 for j in range(num_batteries) 
                 for i in range(num_points))
    
    m.addConstrs(battery_discharge[j, i] - battery_charge[j, i] + 1 == battery_decision[j, i] 
                 for j in range(num_batteries) 
                 for i in range(num_points))
    
# SOC of battery constraints

    m.addConstrs(battery_energy[j, 0] == battery_dict[j]['capacity kWh'] 
                 + battery_charge[j, 0] * battery_dict[j]['max power kW'] * sample_time_h
                     - battery_discharge[j, 0] * battery_dict[j]['max power kW'] * sample_time_h
                 for j in range(num_batteries))
    
    for i in range(1, num_points):
        m.addConstrs(battery_energy[j, i] <= battery_dict[j]['capacity kWh'] for j in range(num_batteries))
        m.addConstrs(battery_energy[j, i] >= 0 for j in range(num_batteries))
        m.addConstrs(battery_energy[j, i] == battery_energy[j, i - 1] 
                     + battery_charge[j, i] * battery_dict[j]['max power kW'] * sample_time_h
                     - battery_discharge[j, i] * battery_dict[j]['max power kW'] * sample_time_h
                     for j in range(num_batteries))

# Sheduled Power Calculations

    m.addConstrs(power_scheduled[i] == gp.quicksum(recurring_active_time[j, i] * recurring_dict[j]['load kW'] * 
                                                   (recurring_dict[j]['No. of small rooms'] 
                                                    + recurring_dict[j]['No. of large rooms']) 
                                                   for j in range(num_recurring_activities)) 
                 for i in range(num_points))
    
# Power at time i Calculation 
    m.addConstrs(Power_time[i] == baseload[i] - solar_power[i] + power_scheduled[i] + 
                 gp.quicksum(battery_power[j, i] for j in range(num_batteries)) for i in range(num_points))

# Room limit constraints

    m.addConstrs(gp.quicksum(recurring_active_time[j, i] * recurring_dict[j]['No. of small rooms'] 
                             for j in range(num_recurring_activities)) <= gp.quicksum(building_dict[k]['No. of small rooms'] 
                                                                                  for k in range(num_buildings)) 
                 for i in range(num_points))
    
    m.addConstrs(gp.quicksum(recurring_active_time[j, i] * recurring_dict[j]['No. of large rooms'] 
                             for j in range(num_recurring_activities)) <= gp.quicksum(building_dict[k]['No. of large rooms'] 
                                                                                  for k in range(num_buildings)) 
                 for i in range(num_points))

# Start Time constraints

    # Should not start at these times
    
    for num in non_start_list:
        m.addConstrs(recurring_start_time[j, num] == 0 for j in range(num_recurring_activities))
        m.addConstrs(recurring_active_time[j, num] == 0 for j in range(num_recurring_activities))
    
    # Should start at the same time every week
    
    for num in range(724, num_points):
        m.addConstrs(recurring_start_time[j, num] == recurring_start_time[j, num - 96 * 7] 
                     for j in range(num_recurring_activities))
        m.addConstrs(recurring_active_time[j, num] == recurring_active_time[j, num - 96 * 7] 
                     for j in range(num_recurring_activities))

    # Should happen only once in a week
    
    for j in range(num_recurring_activities):
        m.addConstr(gp.quicksum(recurring_start_time[j, i] 
                                for i in range(52, 723)) == 1)
        m.addConstr(gp.quicksum(recurring_active_time[j, i] 
                                for i in range(52, 723)) 
                    == recurring_dict[j]['duration'] + 1)
        
    # Should last for required duration hours
    
    for j in range(num_recurring_activities):
        m.addConstrs(recurring_start_time[j, i] <= recurring_active_time[j, i + k] 
                     for k in range(recurring_dict[j]['duration'] + 1) 
                     for i in range(num_points - recurring_dict[j]['duration']))
    
    # Precedence Constraints
    
    for j in range(num_recurring_activities):
        if recurring_dict[j]['No. of precedences'] > 0:
            for k in recurring_dict[j]['precedence list']:
                m.addConstr(gp.quicksum(recurring_start_time[j, i] * day_list[i] 
                                        - recurring_start_time[k, i] * day_list[i] 
                                        for i in range(52, 723)) >= 1) 
    
    # Objective Function
    m.setObjective(max_power + gp.quicksum(0.00025 * Power_time[i] * aemo_price[i]  for i in range(num_points)))
    m.Params.MIPGap = MipGap
    return m

In [17]:
# Main Script 

# User Inputs 

# To define all the instance files to be run and multiplication factor for small and large instances

instance_type = ['small', 'large']
instance = []
multiplication_factor = []

for i in range(5):
    for x in instance_type:
        instance.append(x + '_' + str(i))
        if x == 'small':
            multiplication_factor.append(1.10)
        else:
            multiplication_factor.append(1.15)

# MipGap for the problem 

MipGap = 1 * 10 ** -2

# November 1st is a Sunday thus, starting_day is 7 and start_day is 1

start_day = 1
starting_day = 7


# Current Working Directory 

current_directory = os.getcwd()

# Fixed values for all optimization instances

# Working Hours
    
start_time = 9
stop_time = 17

# Number of days in November
    
num_days = 30

# Constants used while optimizing based on November and Work hours 
    
sample_time_h = 15/60
num_points = int(num_days * 24 / sample_time_h)
num_points_day = int(num_points / num_days)
start_time_sample = int(start_time/sample_time_h)
stop_time_sample = int(stop_time/sample_time_h)

# Obtaining dependencies for the optimization

# Name of the forecast file used 

forecast_file = 'outputs\\forecasting.csv' 

# Prediction Information

forecast_path = current_directory + '\\' + forecast_file

baseload, solar_power = get_forecasts(forecast_path)

# Obtaining AEMO price Information

aemo_path = current_directory + '\\data\\AEMO_price'
    
aemo_price = get_aemo_price(aemo_path)

# Day Lists and Non-start List for optimization

day_list, recurring_non_start_list = day_list_generator(start_day, starting_day)

# Running the optimization for all 10 instances 

# List for storing Gurobi Objects, to retrieve analysis Information

gurobi_object_list = []

for i in range(1):

# Instance path and save file path 

    instance_path = instance_path = current_directory + '\\data\\phase_2_instances_utc\\phase2_instance_' + instance[i] + '.txt'
    
    solution_path = current_directory + '\\outputs\\phase2_instance_solution_' + instance[i] + '.txt'

# Obtaining info for the given instance

    ppoi_dict, building_dict, solar_dict, battery_dict, recurring_dict, once_off_dict, ppoi_keys = get_instance_data(instance_path)
    
    num_buildings = ppoi_dict['No. of buildings']
    
    num_solar_panels = ppoi_dict['No. of pv panels']
    
    num_batteries = ppoi_dict['No. of batteries']
    
    num_recurring_activities = ppoi_dict['No. of recurring activities']
    
    num_once_off_activities = ppoi_dict['No. of once off activities']
    
# First Sub-Problem Solve

    obj = Sub_Problem_1(building_dict, recurring_dict, battery_dict, baseload, solar_power, day_list, recurring_non_start_list)
    
    presolved_model = obj.presolve()
    
    relaxed_presolved_model = presolved_model.relax()
    
    relaxed_presolved_model.optimize()
    
    Objective = relaxed_presolved_model.getObjective()
    
    max_limit = Objective.getValue() * multiplication_factor[i]

    obj.reset()

# Second Sub-Problem Solve using max_limit from previous solution
    
    obj = Sub_Problem_2(building_dict, recurring_dict, battery_dict, baseload, solar_power, MipGap, day_list, recurring_non_start_list, max_limit)
    
    obj.optimize()
    
    gurobi_object_list.append(obj)
    
# Obtaining The solution from The solved problem
    
    recurring_active_time, recurring_start_time, decision = get_optimization_result(obj)
    
# Assigning Rooms to the scheduled tasks

    recurring_dict, total_rooms_assigned = Room_Usage(recurring_dict, building_dict, recurring_start_time)
    
# Saving the solution in the required ".txt" format

    Save_solution(solution_path, recurring_dict, ppoi_dict, total_rooms_assigned, decision)

    obj.reset()


Presolve removed 1459195 rows and 296822 columns
Presolve time: 2.19s
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 17695 rows, 28619 columns and 95774 nonzeros
Model fingerprint: 0x8bd584da
Coefficient statistics:
  Matrix range     [1e-02, 7e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+04]
  RHS range        [1e+00, 7e+02]

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Presolve time: 0.07s
Presolved: 17695 rows, 28619 columns, 95774 nonzeros

Ordering time: 0.01s

Barrier statistics:
 Dense cols : 1
 AA' NZ     : 7.464e+04
 Factor NZ  : 3.879e+05 (roughly 20 MBytes of memory)
 Factor Ops : 1.780e+07 (less than 1 second per iteration)
 Threads    : 6

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   2.04463577e+03 -1.03172